<a href="https://colab.research.google.com/github/InTEGr8or/jupyter-fun/blob/master/nCov2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
try:
  from bs4 import BeautifulSoup
except:
  !pip install beautifulsoup4
  from bs4 import BeautifulSoup
  
import requests
import numpy as np
from dateutil import parser
from datetime import datetime
import pandas as pd

repo = "https://github.com/CSSEGISandData/COVID-19"

tsc_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
tsm_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
tsr_csv = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"

def day_tot(day):
  return f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{day.strftime('%m-%d-%Y')}.csv"

pd.set_option('display.max_rows', 200)

states = {
  'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa',
  'AZ': 'Arizona', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut',
  'DC': 'District of Columbia', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
  'GU': 'Guam', 'HI': 'Hawaii', 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois',
  'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts',
  'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota',
  'MO': 'Missouri', 'MP': 'Northern Mariana Islands', 'MS': 'Mississippi',
  'MT': 'Montana', 'NA': 'National', 'NC': 'North Carolina', 'ND': 'North Dakota',
  'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
  'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon',
  'PA': 'Pennsylvania', 'PR': 'Puerto Rico', 'RI': 'Rhode Island', 'SC': 'South Carolina',
  'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
  'VA': 'Virginia', 'VI': 'Virgin Islands', 'VT': 'Vermont', 'WA': 'Washington',
  'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming', 'AB': 'Alberta',
  'BC': 'British Columbia', 'MB': 'Manitoba', 'NB': 'New Brunswick',
  'NL': 'Newfoundland and Labrador', 'NT': 'Northwest Territories',
  'NS': 'Nova Scotia', 'NU': 'Nunavut', 'ON': 'Ontario', 'PE': 'Prince Edward Island',
  'QC': 'Quebec', 'SK': 'Saskatchewan', 'YT': 'Yukon'
}

def cdate(date):
  return date.strftime("%#m/%#d/%y")

def update_firsts(df, firsts_col):
  # Get date of first death or days since first death.
  df[f'{firsts_col}'] = ''
  dates = sorted([parser.parse(col) for col in df.columns if is_date(col) ])
  # print(f"{firsts_col} dates:", dates, df.columns)
  for i, row in df.iterrows():
    found_first = False
    for date in dates:
      try:
        if not found_first and row[cdate(date)] > 0:
          found_first = True
          df.at[i, firsts_col] = cdate(date)
          # print(f"update_firsts(df, {firsts_col}):\n", date, row[date])
      except Exception as e:
        # pass
        print(f"### ERROR update_firsts(df, {firsts_col}):\n", cdate(date), row, e)
  # print(f"update_firsts({firsts_col})\n", df.iloc[:3])

def is_date(text):
  try:
    s = parser.parse(text)
    return True
  except:
    return False

def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    heat = str(hex(min(int(val.replace('%', '')) * 10 + 56, 255))).split('x')[-1].upper()
    color = f'#{heat}5555'
    # print(val, heat, color)
    return 'color: %s' % color

In [92]:
df = pd.read_csv(tsc_csv).drop(columns=['Lat', 'Long'])
dates = [col for i, col in enumerate(df.columns) if is_date(col) ]
dates.reverse()

dfr = pd.read_csv(tsr_csv)
dfm = pd.read_csv(tsm_csv).drop(columns=['Lat', 'Long'])
df.rename(columns={'Country/Region': 'Country', 'Province/State': 'State'}, inplace=True)
dfm.rename(columns={'Country/Region': 'Country', 'Province/State': 'State'}, inplace=True)
df['Country'].replace('Mainland China', 'China', inplace=True)
df['Country'].replace('United Arab Emirates', 'UAE', inplace=True)
dfm['Country'].replace('Mainland China','China').replace('United Arab Emirates', 'UAE', inplace=True)
mdates = [col for i, col in enumerate(dfm.columns) if is_date(col)]

update_firsts(df, 'First Confirmed')
update_firsts(dfm, 'First Death')

dfm.set_index(['Country', 'State'], inplace=True)
df.fillna('')
df['Country'].fillna('')
dfm.fillna('')

df.set_index(['Country', 'State'], inplace=True)
df['Death Toll'] = 0
df['Death Aging'] = 0

# df['Confirmed Aging'] = datetime.now() - df['First Case'].apply(lambda date: parser.parse(date, fuzzy=True))
# df['Confirmed Aging'] = df['Confirmed Aging'].apply(lambda days: str(days).split(' ')[0])
# df.drop(columns=['First Case'], inplace=True)
# df.sort_values(by=['Country','State'], ascending=[True, True], inplace=True)
# today = dates[0]
# mtoday = mdates[-1]
# print("Latest sheet dates:", today, mtoday)
# for i, row in df.iterrows():
#   country = row['Country']
#   state = row['State']
#   locality = ''
#   ### They've cleaned up their join columns a bit.
#   # if ',' in state:
#   #   locality = state.split(',')[0].strip()
#   #   state = states[state.split(',')[1].strip()]
#   if state == '' or state == 'NaN':
#     try:
#       mrow = dfm.loc[country, mtoday]
#       if 'Phil' in country:
#         # print('Phil:', country, type(country))
#         # print('Phil', mrow[0])
#         mrow = mrow[0]
#     except:
#       mrow = 0
#   else:
#     try:
#       mrow = dfm.loc[country, :].loc[state, mtoday]
#     except:
#       print("ERROR:", country, state, type(state))
#       mrow = 0
#   if isinstance(mrow, (int, float, complex)) and not isinstance(mrow, bool):
#     if mrow > 0:
#       print(row['Country'], row['State'], mrow)
#       df.at[i, 'Death Toll'] = mrow
# print(dfm[mtoday])
# # Remove early results
# for i, date in enumerate(dates):
#   if i < len(dates) -1 and parser.parse(date).day == parser.parse(dates[i + 1]).day:
#     del dates[i + 1]
# # Append to columns
percents = []
for i, date in enumerate(dates):
  d = parser.parse(date)
  col = d.strftime('%B')[:3] + d.strftime('%d')
  df[col] = df[date].replace(np.inf, 0).fillna(0).astype(int)
  if i < len(dates) - 1:
    pcol = dates[i + 1]
    percents.append(col + '%')
    df[col + '%'] = round((df[date] / df[dates[i + 1]].fillna(0) * 100) - 100).replace(np.inf, 0).fillna(0).astype(int).astype(str) + '%'


In [93]:
# df.set_index('Country', inplace=True)
df.style.set_table_styles(
    [{'selector': 'tr:hover',
      'props': [('background-color', 'black')]}]
).applymap(color_negative_red, subset=percents)
df

1        1   
            British Columbia                    0        0        1        1   
US          Orange, CA                          1        1        1        1   
            Los Angeles, CA                     1        1        1        1   
Australia   New South Wales                     3        4        4        4   
            Victoria                            1        1        1        1   
            Queensland                          0        0        0        1   
Cambodia    NaN                                 0        1        1        1   
Sri Lanka   NaN                                 0        1        1        1   
Germany     NaN                                 0        1        4        4   
Finland     NaN                                 0        0        0        1   
UAE         NaN                                 0        0        0        4   
Philippines NaN                                 0        0        0        0   
India       NaN                                 0        0        0        0   
Canada      London, ON                          0        0        0        0   
Italy       NaN                                 0        0        0        0   
UK          NaN                                 0        0        0        0   
Russia      NaN                                 0        0        0        0   
Sweden      NaN                                 0        0        0        0   
US          Santa Clara, CA                     0        0        0        0   
Spain       NaN                                 0        0        0        0   
Australia   South Australia                     0        0        0        0   
US          Boston, MA                          0        0        0        0   
            San Benito, CA                      0        0        0        0   
Belgium     NaN                                 0        0        0        0   
US          Madison, WI                         0        0        0        0   
Others      Diamond Princess cruise ship        0        0        0        0   
US          San Diego County, CA                0        0        0        0   
            San Antonio, TX                     0        0        0        0   
Egypt       NaN                                 0        0        0        0   

                                          1/30/20  1/31/20  ...  Jan27%  \
Country     State                                           ...           
China       Anhui                             200      237  ...     17%   
            Beijing                           114      139  ...     18%   
            Chongqing                         182      211  ...     47%   
            Fujian                            101      120  ...     69%   
            Gansu                              26       29  ...    100%   
            Guangdong                         354      436  ...     36%   
            Guangxi                            78       87  ...     28%   
            Guizhou                            12       29  ...     40%   
            Hainan                             46       52  ...     50%   
            Hebei                              65       82  ...     38%   
            Heilongjiang                       44       59  ...     40%   
            Henan                             278      352  ...     54%   
            Hubei                            4903     5806  ...     34%   
            Hunan                             277      332  ...     45%   
            Inner Mongolia                     19       20  ...     57%   
            Jiangsu                           129      168  ...     42%   
            Jiangxi                           162      240  ...    100%   
            Jilin                              14       14  ...     50%   
            Liaoning                           41       48  ...     29%   
            Ningxia                            17       21  ...     75%   
            Qin